In [23]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns

sns.set_style('white')
sns.set_context("poster")

In [2]:
# IMPORT ZO
zo = pd.read_csv('zero-offset.csv')

In [3]:
# IMPORT DATA
data = pd.read_csv('data1.csv')
# Alter date
data['timestamp'] = '2016-01-10 20:00:00'

In [4]:
# CONVERT timestamps to correct data time
zo.start_timestamp = pd.to_datetime(zo['start_timestamp'], format='%Y-%m-%dT%H:%M:%S')
data.timestamp = pd.to_datetime(data['timestamp'], format='%Y-%m-%dT%H:%M:%S')

# SORT BY time df.sort('Date')
data = data.sort_values('timestamp')
zo = zo.sort_values('start_timestamp')

# REMOVE ms
zo.start_timestamp = zo.start_timestamp.apply(lambda x: x.replace(microsecond=0))

In [5]:
# Drop unnecessary columns
zo = zo.reset_index()
data = data.reset_index()
zo = zo.drop(['Unnamed: 0', 'index'], axis=1)
data = data.drop(['id', 'index'], axis=1)
# it's Unnamed: 0 and 'index' in the real ones

In [6]:
# Rename columns
zo.columns = ['sensor_id', 'offset_value', 'timestamp']

In [19]:
# Testing fillna (ffill)
test_data = pd.DataFrame({'timestamp': ['2016-01-10 19:59:58', '2016-01-10 19:59:59', '2016-01-10 20:00:00'],
                          'sensor_id': [166, 166, 166],
                          'd_val': [50, 60, 70]})
test_zo = pd.DataFrame({'timestamp': ['2016-01-10 19:59:59'],
                     'sensor_id': [166],
                     'offset_value': [20]})
test_data.timestamp = pd.to_datetime(test_data['timestamp'], format='%Y-%m-%dT%H:%M:%S')
test_zo.timestamp = pd.to_datetime(test_zo['timestamp'], format='%Y-%m-%dT%H:%M:%S')

df = pd.merge(test_data, test_zo, how='left', on=['timestamp', 'sensor_id'])
df = df.fillna(method='ffill')  # forward fill (should fill na with previous value)
df.head()

,d_val,sensor_id,timestamp,offset_value
0,50,166,2016-01-10 19:59:58,NaN
1,60,166,2016-01-10 19:59:59,20.0
2,70,166,2016-01-10 20:00:00,20.0


In [17]:
df = pd.merge(data, zo, how='left', on=['timestamp', 'sensor_id'])
df = df.fillna(method='ffill')  # forward fill (should fill na with previous value)
print(df.head())
print(df.tail())
len(df)
# Join by LEFT

            timestamp  sensor_id   d_st      d_val  offset_value
0 2016-01-10 20:00:00        166  10001  1025.7030           NaN
1 2016-01-10 20:00:00        198   1101   645.8704           NaN
2 2016-01-10 20:00:00        199   1101   669.6980           NaN
3 2016-01-10 20:00:00        200   1101   994.8445           NaN
4 2016-01-10 20:00:00        201   1101   760.4407           NaN
                 timestamp  sensor_id  d_st     d_val  offset_value
999995 2016-01-10 20:00:00       2752  1101  15.60091           NaN
999996 2016-01-10 20:00:00       3263  1001  12.67423           NaN
999997 2016-01-10 20:00:00       3264  1001  12.58104           NaN
999998 2016-01-10 20:00:00       3266  1001  12.66805           NaN
999999 2016-01-10 20:00:00       3263  1001  12.69028           NaN


1000000

In [20]:
zo.head()

,sensor_id,offset_value,timestamp
0,60,971.59,2016-01-10 19:57:57
1,170,0.00,2016-01-10 19:57:57
2,103,0.00,2016-01-10 19:57:57
3,219,407.43,2016-01-10 19:57:57
4,171,0.00,2016-01-10 19:57:57


In [21]:
zo.tail()

,sensor_id,offset_value,timestamp
33778,221,-228.558044,2017-04-11 21:49:10
33779,108,0.000000,2017-04-11 21:49:10
33780,220,-621.843943,2017-04-11 21:49:10
33781,263,0.000000,2017-04-11 21:49:10
33782,3003,0.000000,2017-04-11 21:49:10


In [33]:
unique_id = zo.sensor_id.unique()
unique_id

array([  60,  170,  103,  219,  171,  172,   72,  102,  173,  174,  101,
        218,  249,  175,  217,  100,  264,   73,  176,  177,   99,  216,
        178,   98,  179,  215,   74,   97,  248,  180,  250,  241,  104,
         71,  111,  159,  252,  224,  160,  110,  262,  161,  109,   69,
        162,  163,  223,  108,  164,  222,  107,  251,  165,  270,   70,
        166,  106,  167,  168,  105,  263,  220,  240,  169,  181,   96,
        182,   88,  194,  245,  195,   87,  196,  208,  197,   86,   79,
        198,   85,  267,  199,  207,   84,  244,  200,  268,   80,  201,
         83,  206,  202,  203,  205,  243,   82,  204,   78,  209,  193,
         89,  214,  183,  269,   75,   95,  265,  184,   94,  247,  185,
        213,   93,  186,   76,   68,  187,  188,  212,  189,   91,  211,
        246,  190,   77,   90,  266,  191,  210,  242,  192,   92,  158,
        221,   81,  125,  119,  147,   62,  271,   65,  118,  148,  137,
        234,  149,  229,  136,  117,  254,  150,  1

In [37]:
counter = 0

for uni_id in unique_id:
    length = len(zo[zo.sensor_id == uni_id].offset_value.unique())
    if length > 1:
        counter += 1
print('Number of non-unique offset-value: ' + str(counter))
print('Number of unique offset-value: ' + str((len(unique_id) - counter)))

Number of non-unique offset-value: 108
Number of unique offset-value: 201
